In [4]:
from datasets import load_dataset, Features, ClassLabel, Value

# features = Features({'label': ClassLabel(num_classes=3, names=['POS', 'NEG', 'NEU'], id=None),
#  'title': Value(dtype='string', id=None)})
raw_datasets = load_dataset('csv',data_files='./data/new_dataset.csv', split='train')

copy_of_features = raw_datasets.features.copy()
raw_datasets = raw_datasets.rename_column('title', 'text')
raw_datasets = raw_datasets.train_test_split(0.2,0.8)

Using custom data configuration default-3597555db7518180
Found cached dataset csv (/home/cerdamara/.cache/huggingface/datasets/csv/default-3597555db7518180/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [5]:
copy_of_features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'id_news': Value(dtype='int64', id=None),
 'title': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "finiteautomata/beto-sentiment-analysis"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file config.json from cache at /home/cerdamara/.cache/huggingface/hub/models--finiteautomata--beto-sentiment-analysis/snapshots/2d232b7b937ca0f6940f6b32ce5aaaeb012d8b38/config.json
Model config BertConfig {
  "_name_or_path": "finiteautomata/beto-sentiment-analysis",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "tr

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:00<00:00,  6.90ba/s]


In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id_news', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 262
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id_news', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 66
    })
})

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(262))

small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(66))

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",label_names = ['POS','NEG','NEU'])


from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id_news, Unnamed: 0, text. If id_news, Unnamed: 0, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 262
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 99


ValueError: too many dimensions 'str'